In [222]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import sqlite3
import string
import numpy as np

import pyLDAvis
import pyLDAvis.gensim

In [223]:
stop = stopwords.words('english')
stop.extend(['I', 's', 'na', 'oh', 'la', 'like', 'go', 'let'])

In [224]:
conn = sqlite3.connect("blink.db")
df = pd.read_sql_query("select * from Songs;", conn)
df.shape
df['album_name'].unique()

array(['Flyswatter', 'Buddha', 'Cheshire Cat', 'Dude Ranch',
       'Enema of the State', 'Take Off Your Pants and Jacket',
       'Blink-182', 'Neighborhoods', 'None', 'California', 'Nine'],
      dtype=object)

In [225]:
test = df['lyrics'][1]
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
test.translate(translator)

'When the clock strikes two There s so much to do And I cant explain what I need  Jobs and social groups Hearing the latest news Keeping your reputation clean  And I don t wanna worry About being on time I see the way you hurry And time runs your life again  The difference between east and west Money means so much less And objects aren t so important to buy  I wish that earth clock Could often just be stopped And then we look into the time  And I don t wanna worry About being on time I see the way you hurry And time runs your life again'

In [226]:
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))

def remove_punctuation(lyrics):
    return lyrics.lower().translate(translator)

def remove_stopwords(lyrics):
    return ' '.join([x for x in remove_punctuation(lyrics).split() if x not in stop])

def clean_lyrics(lyrics):
    return remove_stopwords(remove_punctuation(lyrics))

In [227]:
blink_corpus = [clean_lyrics(x) for x in df['lyrics'].tolist()]

In [228]:
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim

In [237]:
blink_tokens = [word_tokenize(x) for x in blink_corpus]

dictionary = corpora.Dictionary(blink_tokens)
dictionary.filter_extremes(no_below=2)
corpus = [dictionary.doc2bow(text) for text in blink_tokens]

%time ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=10)

CPU times: user 1.48 s, sys: 0 ns, total: 1.48 s
Wall time: 1.48 s


In [238]:
for i,topic in ldamodel.show_topics(formatted=True, num_topics=3, num_words=10):
    print(str(i)+": "+ topic)
    print()

0: 0.026*"time" + 0.014*"last" + 0.013*"one" + 0.012*"never" + 0.012*"need" + 0.011*"home" + 0.009*"got" + 0.009*"know" + 0.009*"makes" + 0.008*"said"

1: 0.010*"never" + 0.010*"love" + 0.009*"left" + 0.009*"home" + 0.009*"cause" + 0.008*"one" + 0.008*"gone" + 0.008*"want" + 0.008*"time" + 0.007*"night"

2: 0.018*"know" + 0.016*"na" + 0.013*"say" + 0.012*"back" + 0.011*"time" + 0.011*"wan" + 0.011*"wait" + 0.010*"away" + 0.010*"get" + 0.010*"night"



In [239]:
def song_classification(idx):
    try:
        return max(ldamodel[corpus[idx]],key=lambda item:item[1])[0]
    except:
        print("Out of index range")

df['classification'] = np.vectorize(song_classification)(np.arange(175))

In [240]:
df['album_name'].unique()

array(['Flyswatter', 'Buddha', 'Cheshire Cat', 'Dude Ranch',
       'Enema of the State', 'Take Off Your Pants and Jacket',
       'Blink-182', 'Neighborhoods', 'None', 'California', 'Nine'],
      dtype=object)

In [241]:
album_breakdown = df[['classification', 'album_name']].groupby('album_name')['classification'].value_counts(normalize=True)

In [242]:
df.to_csv('album_breakdown.csv', index = False)

In [243]:
%matplotlib inline

vis = pyLDAvis.gensim.prepare(topic_model=ldamodel, corpus=corpus, dictionary=dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

/home/nick/miniconda3/envs/py36/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [245]:
pyLDAvis.save_html(vis, 'blink_lda.html')